In [6]:
%matplotlib inline
import pandas as pd
# from sklearn import 
import numpy as np
import os

In [7]:
filepath_X_train = os.path.join(os.path.abspath('..'), 'Files', 'AMF_train_X.csv')
filepath_y_train = os.path.join(os.path.abspath('..'), 'Files', 'AMF_train_Y.csv')
filepath_X_test = os.path.join(os.path.abspath('..'), 'Files', 'AMF_train_X.csv')


X_train = pd.read_csv(filepath_X_train)
y_train = pd.read_csv(filepath_y_train)
X_test = pd.read_csv(filepath_X_test)